In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Carga de los datos de entrenamiento y validacion

In [2]:
train_dir = '../../dataset/entrenamiento'
test_dir = '../../dataset/test'

datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normaliza los valores de píxeles al rango [0, 1]
    validation_split=0.25  # Porcentaje de datos para validación
)

batch_size = 32  # Tamaño de lote.
image_size = (100, 100)  # tamaño de las imagenes para ser ajustada.

train_data = datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Para clasificación con 5 clases
    subset='training'
)

validation_data = datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 14528 images belonging to 16 classes.
Found 4832 images belonging to 16 classes.


In [3]:
total_data_train = train_data.samples
total_data_validation = validation_data.samples

print("Total de datos de entrenamiento:", total_data_train)
print("Total de datos de validación:", total_data_validation)

total_clases = train_data.num_classes

print("Total de clases:", total_clases)

Total de datos de entrenamiento: 14528
Total de datos de validación: 4832
Total de clases: 16


## Implementacion del modelo

In [12]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dropout, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

input_shape = (100, 100, 3)
# Cargar el modelo preentrenado DenseNet121 sin la capa superior 
# (incluyendo pesos)
dn_base_model = DenseNet121(weights='imagenet', 
                            include_top=False, 
                            input_shape=input_shape)

# Agregar capas personalizadas para la clasificación
dn_x = dn_base_model.output
dn_x = GlobalAveragePooling2D()(dn_x)
dn_x = Dense(1024, activation='relu')(dn_x)

# Después de la primera capa densa
dn_x = Dense(1024, activation='relu', kernel_regularizer=l2(0.001))(dn_x)
dn_x = Dropout(0.5)(dn_x)

# Después de la segunda capa densa
dn_x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(dn_x)
dn_x = Dropout(0.4)(dn_x)

# Después de la tercera capa densa
dn_x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(dn_x)
dn_x = Dropout(0.3)(dn_x)

dn_predictions = Dense(total_clases, activation='softmax')(dn_x)

# Crear el modelo completo
modeloDenseNet = Model(inputs=dn_base_model.input, outputs=dn_predictions)

# Congelar las capas del modelo base (DenseNet121) 
# para no entrenarlas nuevamente
for layer in dn_base_model.layers:
    layer.trainable = False

## Compilacion de los modelos

In [13]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
# Compilado del modelo DenseNet
modeloDenseNet.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy', Precision(), Recall()])

modeloDenseNet.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 106, 106, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 50, 50, 64)   9408        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 50, 50, 64)   256         conv1/conv[0][0]                 
____________________________________________________________________________________________

## Entrenamiento del modelo

In [14]:
from tensorflow.keras.callbacks import TensorBoard

# Calculo de la cantidad de datos a usar por epoca
steps_per_epoch_train = total_data_train // batch_size
steps_per_epoch_validation = total_data_validation // batch_size
epocas = 100

print("epocas: ", epocas)
print("Pasos por epoca para entrenar: ", steps_per_epoch_train)
print("Pasos por epoca para validar: ", steps_per_epoch_validation)

epocas:  80
Pasos por epoca para entrenar:  454
Pasos por epoca para validar:  151


In [15]:
from tensorflow.keras.callbacks import EarlyStopping

boardModeloDN = TensorBoard(log_dir='logs/denseNetFinal80Epochs')
# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

# Entrenamiento del modelo DenseNet
modeloDenseNet.fit(
    train_data,
    epochs=epocas, batch_size=batch_size,
    validation_data=validation_data,
    steps_per_epoch=steps_per_epoch_train,
    validation_steps=steps_per_epoch_validation,
    callbacks=[early_stopping, boardModeloDN]
)

Epoch 1/80
454/454 [==============================] - 23s 42ms/step - loss: 4.0476 - accuracy: 0.3722 - precision_1: 0.6376 - recall_1: 0.1934 - val_loss: 2.5600 - val_accuracy: 0.8011 - val_precision_1: 0.8498 - val_recall_1: 0.7707
Epoch 2/80
454/454 [==============================] - 17s 38ms/step - loss: 2.2736 - accuracy: 0.8676 - precision_1: 0.9041 - recall_1: 0.8295 - val_loss: 2.3854 - val_accuracy: 0.8324 - val_precision_1: 0.8575 - val_recall_1: 0.8166
Epoch 3/80
454/454 [==============================] - 17s 38ms/step - loss: 1.9387 - accuracy: 0.9371 - precision_1: 0.9493 - recall_1: 0.9234 - val_loss: 2.3322 - val_accuracy: 0.8342 - val_precision_1: 0.8509 - val_recall_1: 0.8257
Epoch 4/80
454/454 [==============================] - 17s 38ms/step - loss: 1.7500 - accuracy: 0.9640 - precision_1: 0.9692 - recall_1: 0.9572 - val_loss: 2.2558 - val_accuracy: 0.8435 - val_precision_1: 0.8580 - val_recall_1: 0.8363
Epoch 5/80
454/454 [==============================] - 17s 38ms/s

Epoch 36/80
454/454 [==============================] - 17s 38ms/step - loss: 0.0454 - accuracy: 0.9988 - precision_1: 0.9988 - recall_1: 0.9988 - val_loss: 0.7693 - val_accuracy: 0.8589 - val_precision_1: 0.8680 - val_recall_1: 0.8516
Epoch 37/80
454/454 [==============================] - 17s 38ms/step - loss: 0.0427 - accuracy: 0.9988 - precision_1: 0.9989 - recall_1: 0.9985 - val_loss: 0.8213 - val_accuracy: 0.8351 - val_precision_1: 0.8453 - val_recall_1: 0.8303
Epoch 38/80
454/454 [==============================] - 17s 38ms/step - loss: 0.0414 - accuracy: 0.9990 - precision_1: 0.9991 - recall_1: 0.9988 - val_loss: 0.8079 - val_accuracy: 0.8406 - val_precision_1: 0.8508 - val_recall_1: 0.8334
Epoch 39/80
454/454 [==============================] - 17s 38ms/step - loss: 0.0374 - accuracy: 1.0000 - precision_1: 1.0000 - recall_1: 1.0000 - val_loss: 0.6909 - val_accuracy: 0.8584 - val_precision_1: 0.8724 - val_recall_1: 0.8520
Epoch 40/80
454/454 [==============================] - 18s 3

## Evaluacion de los modelos

In [ ]:
# Instalar en caso no se cuente con la libreria.
pip install scikit-learn

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

# Crea un generador de datos para el conjunto de prueba
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Crea el generador de datos de prueba
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # No es necesario barajar el conjunto de prueba
)

Found 156 images belonging to 16 classes.


In [17]:
%run evaluador_modelos.ipynb

In [18]:
ModelEvaluator(modeloDenseNet, test_generator, 'modeloDenseNet').evaluate()

----------Resultados del modelo: modeloDenseNet ------
Accuracy: 0.6794871794871795
Precision: 0.7340711283018975
Recall: 0.6794871794871795
F1 Score: 0.6613151793966067
Confusion Matrix:
[[10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 4  5  0  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 4  0  6  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  9  0  0  1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  9  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  6  0  0  0  0  1  1  2  0  0  0]
 [ 0  0  0  0  0  0 10  0  0  0  0  0  0  0  0  0]
 [ 1  0  1  0  0  1  4  2  0  0  0  0  1  0  0  0]
 [ 0  0  0  0  3  0  0  0  7  0  0  0  0  0  0  0]
 [ 0  0  0  3  1  2  0  0  0  2  0  1  1  0  0  0]
 [ 0  0  0  1  0  1  0  0  0  0  3  0  0  1  0  0]
 [ 0  0  0  0  2  0  0  0  1  1  0  6  0  0  0  0]
 [ 0  0  0  0  0  3  0  0  0  0  0  1  5  1  0  0]
 [ 0  0  0  1  0  0  0  0  1  0  0  0  0  8  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  9  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  1  0 

In [20]:
# Cargar la extension de tensorboard de colab
%load_ext tensorboard

In [ ]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

In [21]:
# Ejecutar tensorboard e indicarle que lea la carpeta "logs"
%tensorboard --logdir logs

## Exportacion del modelo

In [19]:
modeloDenseNet.save('modelo_final.h5')